In [263]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from jupyterthemes import jtplot
import requests
import json
jtplot.style()

# Problem 1: Burglaries

A dataset containing details about Metro Nashville Police Department reported incidents is available at https://data.nashville.gov/Police/Metro-Nashville-Police-Department-Incidents/2u6v-ujjs. Make use of the API to find all aggravated burglary incidents that were reported during the six month period from January 1, 2021 through June 30, 2021.

In [264]:
endpoint = "https://data.nashville.gov/resource/2u6v-ujjs.geojson?"
offset = 0
limit = 10000
burg = gpd.GeoDataFrame()
while True:
    params1 = {
        "offense_description":"BURGLARY- AGGRAVATED",
        #offense_nibrs":220,
        "$where":"incident_reported between '2021-01-01T00:00:00.000' and '2021-06-30T00:00:00.000'",
        "$offset":offset,
        "$limit":limit
    }
    res = requests.get(endpoint, params1)
    res_gdf = gpd.read_file(res.text)
    burg = gpd.GeoDataFrame(pd.concat([burg, res_gdf], ignore_index = True))
    
    if len(res_gdf) != limit:
        break
        
    offset += limit

burg = burg.drop_duplicates(subset = 'incident_number')
burg

,victim_county_resident,zip_code,victim_number,offense_nibrs,rpa,latitude,victim_race,incident_number,investigation_status,offense_number,...,weapon_primary,report_type_description,victim_type,incident_status_code,incident_status_description,zone,victim_gender,incident_occurred,primary_key,geometry
0,RESIDENT,None,1,220,8203,36.15,B,20210249540,Open,1,...,09,DISPATCHED,I,O,OPEN,511,F,2021-05-04T23:45:00,20210249540_11,POINT (-86.77000 36.15000)
1,RESIDENT,None,1,220,9615,36.2,W,20210248840,Open,1,...,09,DISPATCHED,I,O,OPEN,525,F,2021-05-04T12:06:00,20210248840_11,POINT (-86.63000 36.20000)
2,NON RESIDENT,None,1,220,6901,36.14,W,20210244919,Open,1,...,17,DISPATCHED,I,O,OPEN,815,M,2021-05-01T23:00:00,20210244919_11,POINT (-86.78000 36.14000)
3,NON RESIDENT,None,3,220,None,36.14,W,20210244892,Open,1,...,17,DISPATCHED,I,O,OPEN,None,M,2021-05-01T21:30:00,20210244892_13,POINT (-86.78000 36.14000)
5,RESIDENT,None,2,220,8401,36.11,W,20210248402,Open,1,...,17,DISPATCHED,I,O,OPEN,313,F,2021-05-04T00:00:00,20210248402_12,POINT (-86.73000 36.11000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1041,RESIDENT,37013,1,220,8861,36.052,B,20210029313,Closed,1,...,01,DISPATCHED,I,A,CLEARED BY ARREST,327,F,2021-01-15T21:50:00,20210029313_11,POINT (-86.64700 36.05200)
1042,RESIDENT,37210,1,220,8201,36.152,B,20210073787,Closed,1,...,09,DISPATCHED,I,A,CLEARED BY ARREST,511,F,2021-02-06T12:00:00,20210073787_11,POINT (-86.76500 36.15200)
1043,RESIDENT,37207,1,220,3013,36.204,B,20210127915,Closed,1,...,17,DISPATCHED,I,A,CLEARED BY ARREST,631,M,2021-03-03T15:30:00,20210127915_11,POINT (-86.80700 36.20400)
1044,None,None,1,220,8655,36.05,None,20210126551,Open,1,...,17,None,B,O,OPEN,827,None,2021-02-23T10:30:00,20210126551_11,POINT (-86.72000 36.05000)


# Problem 2: Census Tract Shapefiles

Download the 2019 census tract shapefiles for Tennessee from https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2019.html. (The FIPS code for Tennessee is 47). Perform a spatial join to determine the census tract in which each burglary incident occurred. Which census tract had the highest number of burglaries? Warning - each incident can appear multiple times if there are multiple victims, so be sure that you aren't double-counting any incidents.

In [265]:
tn_tracts = gpd.read_file("data/tl_2019_47_tract.shp").to_crs(epsg = 4326)

In [266]:
tn_tracts.columns = tn_tracts.columns.str.lower()

In [267]:
tn_tracts = (tn_tracts.rename(columns = {'statefp':'state',
                            'countyfp':'county',
                            'tractce':'tract'}
                )
                .drop(columns = ['name', 'namelsad'])
            )

In [268]:
tn_bur = gpd.sjoin(burg, tn_tracts, op = 'within')

In [269]:
tn_bur.value_counts('tract')

tract
016300    53
016200    21
011001    20
019500    19
019300    19
          ..
011900     1
018700     1
015405     1
018601     1
015627     1
Length: 147, dtype: int64

# Problem 3: Populations

For this part, you'll need to request a census API key. Using the 2019 American Community Survey API, obtain, for each census tract, the population (B01001_001E in the detailed tables) and the median income (S1901_C01_012E in the subject tables). Hint: Tennessee's FIPS code is 47 and Davidson County's FIPS code is 37. Merge this new data with the burglaries data above.

In [270]:
with open('C:/Users/jrior/Documents/NSS/api_keys.json') as fi:
    credentials = json.load(fi)
    
api_key = credentials['census_data']

In [271]:
endpoint = "https://api.census.gov/data/2019/acs/acs5?"
query = f"get=NAME,B01001_001E&for=tract:*&in=state:47"#&key={api_key}"
response2 = requests.get(endpoint+query)
response2

<Response [200]>

In [272]:
res2_dict = response2.json()
tn_pops = pd.DataFrame().from_dict(res2_dict)
tn_pops.columns = tn_pops.loc[0]
tn_pops = (tn_pops.drop(tn_pops.index[0])
           .reset_index(drop = True)
           .rename(columns = {'NAME': 'name',
                              'B01001_001E': 'population'}
                  )
          )

In [273]:
endpoint = "https://api.census.gov/data/2019/acs/acs5/subject?"
query = f"get=NAME,S1901_C01_012E&for=tract:*&in=state:47"#&key={api_key}"
response3 = requests.get(endpoint+query)
response3

<Response [200]>

In [274]:
res3_dict = response3.json()
tn_med_inc = pd.DataFrame().from_dict(res3_dict)
tn_med_inc.columns = tn_med_inc.loc[0]
tn_med_inc = (tn_med_inc.drop(tn_med_inc.index[0])
              .reset_index(drop = True)
              .rename(columns = {'NAME': 'name',
                                 'S1901_C01_012E': 'median_income'}
                     )
             )

In [275]:
tn = tn_pops.merge(tn_med_inc)
tn[['county_name', 'state_name']] = (tn['name'].str.split(pat = ', ',
                             expand = True
                            )
               .drop(columns = [0])
    )
tn = tn.drop(columns = 'name')

In [276]:
tn_burgs = tn.merge(tn_bur)

In [278]:
tn_burgs[tn_burgs['tract'] == '016300']

,population,state,county,tract,median_income,county_name,state_name,victim_county_resident,zip_code,victim_number,...,primary_key,geometry,index_right,geoid,mtfcc,funcstat,aland,awater,intptlat,intptlon
356,2560,47,037,016300,50710,Davidson County,Tennessee,NON RESIDENT,None,6,...,20210240530_16,POINT (-86.79000 36.15000),677,47037016300,G5020,S,1099214,0,+36.1431906,-086.7899132
357,2560,47,037,016300,50710,Davidson County,Tennessee,NON RESIDENT,None,3,...,20210232671_13,POINT (-86.79000 36.15000),677,47037016300,G5020,S,1099214,0,+36.1431906,-086.7899132
358,2560,47,037,016300,50710,Davidson County,Tennessee,RESIDENT,None,5,...,20210203105_25,POINT (-86.79000 36.14000),677,47037016300,G5020,S,1099214,0,+36.1431906,-086.7899132
359,2560,47,037,016300,50710,Davidson County,Tennessee,RESIDENT,None,1,...,20210035296_11,POINT (-86.79000 36.14000),677,47037016300,G5020,S,1099214,0,+36.1431906,-086.7899132
360,2560,47,037,016300,50710,Davidson County,Tennessee,NON RESIDENT,None,1,...,20210212785_11,POINT (-86.79000 36.14000),677,47037016300,G5020,S,1099214,0,+36.1431906,-086.7899132
361,2560,47,037,016300,50710,Davidson County,Tennessee,NON RESIDENT,None,1,...,20210183062_11,POINT (-86.79000 36.14000),677,47037016300,G5020,S,1099214,0,+36.1431906,-086.7899132
362,2560,47,037,016300,50710,Davidson County,Tennessee,RESIDENT,None,1,...,20210065567_11,POINT (-86.79000 36.14000),677,47037016300,G5020,S,1099214,0,+36.1431906,-086.7899132
363,2560,47,037,016300,50710,Davidson County,Tennessee,RESIDENT,None,1,...,20210146767_11,POINT (-86.79000 36.14000),677,47037016300,G5020,S,1099214,0,+36.1431906,-086.7899132
364,2560,47,037,016300,50710,Davidson County,Tennessee,NON RESIDENT,None,1,...,20210187464_11,POINT (-86.79000 36.14000),677,47037016300,G5020,S,1099214,0,+36.1431906,-086.7899132
365,2560,47,037,016300,50710,Davidson County,Tennessee,RESIDENT,None,1,...,20210160726_11,POINT (-86.79000 36.14000),677,47037016300,G5020,S,1099214,0,+36.1431906,-086.7899132


# Problem 4

Create a choropleth showing the number of burglaries per 1000 residents for each census tract.

# Problem 5

Finally, we'll build some statistical models to see how well we can explain the number of aggravated burglaries using the median income of each census tract. For this, we'll be using the Generalized Linear Models module of the statsmodels library.

a. Build a "base model" - a Poisson regression model with just an intercept term with target variable the rate of burglaries per census tract. (Offset using the [log of the] population so that we are looking at the rate of burglaries per population instead of the number of burglaries.)

b. Now, build a Poisson regression model with target variable the rate of burglaries and predictor variable the median income. (Don't forget to offset by the population).

c. Finally, try out a negative binomial model. To get started with a negative binomial model, you can check out this tutorial.

d. How do your models compare? Hint: the fit models have an AIC attribute.

# Problem 6: Presentation